# Word chains

"Word chain" puzzles are where you transform one word into another, by changing one letter at a time, with all the intermediate steps being valid words. 

For instance, you can transform 'rash' to 'jags' like this:

```
rash
Bash
basS
baGs
Jags
```

(the capital letter is the one changed in each step).

## Part 1

Given this [list of words](words4.txt), what is the minimum number of steps to go from `vice` to `wars`?

In [1]:
import string
import heapq

In [2]:
words = [w.strip() for w in open('08-offices.txt').readlines()]
len(words)

2336

In [3]:
def adjacents(word):
    return [word[0:i] + l + word[i+1:]
           for i in range(len(word))
           for l in string.ascii_lowercase
           if l != word[i]]

In [4]:
neighbours = {w: [n for n in adjacents(w) if n in words]
             for w in words}

In [5]:
def distance(w1, w2):
    return sum(1 for i in range(len(w1))
               if w1[i] != w2[i])

In [6]:
# def extend(chain):
#     return [chain + [s] for s in neighbours[chain[-1]]
#            if s not in chain]

In [7]:
def extend(chain, closed=None):
    if closed:
        nbrs = set(neighbours[chain[-1]]) - closed
    else:
        nbrs = neighbours[chain[-1]]
    return [chain + [s] for s in nbrs
           if s not in chain]

In [8]:
def extend_raw(chain):
    nbrs = [w for w in adjacents(chain[-1]) if w in words]
    return [chain + [s] for s in nbrs
           if s not in chain]

In [9]:
def bfs_search(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            agenda = agenda[1:] + successors
    if agenda:
        return current
    else:
        return None        

In [10]:
def bfs_search_closed(start, goal, debug=False):
    agenda = [[start]]
    closed = set()
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend(current, closed)
            agenda = agenda[1:] + successors
    if agenda:
        return current
    else:
        return None   

In [11]:
def dfs_search(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            agenda = successors + agenda[1:]
    if agenda:
        return current
    else:
        return None        

In [12]:
def astar_search(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if agenda:
        return current
    else:
        return None        

In [13]:
# Uses direct lookup of successors, rather than using cached neighbours in the dict
def astar_search_raw(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend_raw(current) # Difference here
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if agenda:
        return current
    else:
        return None        

In [14]:
def astar_search_closed(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    closed = set()
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend(current, closed)
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if agenda:
        return current
    else:
        return None   

In [15]:
astar_search('vice', 'wars')

['vice', 'dice', 'dire', 'dare', 'ware', 'wars']

In [16]:
astar_search_raw('vice', 'wars')

['vice', 'dice', 'dire', 'dare', 'ware', 'wars']

In [17]:
astar_search_raw('boon', 'sell')

['boon', 'boot', 'bolt', 'belt', 'bell', 'sell']

In [18]:
len(astar_search('vice', 'wars'))

6

In [19]:
# len(bfs_search('vice', 'wars'))

In [20]:
len(bfs_search_closed('vice', 'wars'))

6

In [21]:
len(dfs_search('vice', 'wars'))

793

In [22]:
%%timeit
astar_search('vice', 'wars')

1000 loops, best of 3: 273 µs per loop


In [23]:
%%timeit
astar_search_raw('vice', 'wars')

10 loops, best of 3: 25.6 ms per loop


In [24]:
%%timeit
astar_search_closed('vice', 'wars')

1000 loops, best of 3: 300 µs per loop


In [25]:
# %%timeit
# bfs_search('vice', 'wars')

In [26]:
%%timeit
bfs_search_closed('vice', 'wars')

1 loop, best of 3: 664 ms per loop


In [27]:
%%timeit
dfs_search('vice', 'wars')

10 loops, best of 3: 147 ms per loop


## Part 2

The example shows that `jags` is reachable in four steps from `rash`. There are 11 words one step away from `rash`: 
`bash`, `cash`, `dash`, `gash`, `hash`, `lash`, `mash`, `rasp`, `rush`, `sash`, and `wash`. 

There are 47 words reachable in one or two steps from `rash`. They are `base`, `bash`, `bask`, `bass`, `bast`, `bath`, `bosh`, `bush`, `case`, `cash`, `cask`, `cast`, `dash`, `dish`, `gash`, `gasp`, `gosh`, `gush`, `hash`, `hasp`, `hath`, `hush`, `lash`, `lass`, `last`, `lath`, `lush`, `mash`, `mask`, `mass`, `mast`, `math`, `mesh`, `mush`, `push`, `ramp`, `rasp`, `ruse`, `rush`, `rusk`, `rust`, `sash`, `sass`, `tush`, `wash`, `wasp`, and `wish`.

There are 180 words reachable in up to three steps from `rash`.

How many words are reachable in up to ten steps from `vice`?

In [28]:
def reachable_in(word, n, trim_extras=False):
    reachable = set()
    boundary = set([word])
    for i in range(n):
        extras = set()
        for w in boundary:
            extras.update(neighbours[w])
        if trim_extras:
            extras.difference_update(reachable)
        reachable.update(boundary)
        boundary = extras.copy()
    return reachable.union(extras).difference(set([word]))

In [52]:
neighbours['rash']

['bash',
 'cash',
 'dash',
 'gash',
 'hash',
 'lash',
 'mash',
 'sash',
 'wash',
 'rush',
 'rasp']

In [29]:
len(reachable_in('rash', 1)), ', '.join(sorted('`{}`'.format(r) for r in reachable_in('rash', 1)))

(11,
 '`bash`, `cash`, `dash`, `gash`, `hash`, `lash`, `mash`, `rasp`, `rush`, `sash`, `wash`')

In [47]:
len(reachable_in('rash', 1)), ', '.join(sorted('<code>{}</code>'.format(r) for r in reachable_in('rash', 1)))

(11,
 '<code>bash</code>, <code>cash</code>, <code>dash</code>, <code>gash</code>, <code>hash</code>, <code>lash</code>, <code>mash</code>, <code>rasp</code>, <code>rush</code>, <code>sash</code>, <code>wash</code>')

In [30]:
len(reachable_in('rash', 2)), ', '.join(sorted('`{}`'.format(r) for r in reachable_in('rash', 2)))

(47,
 '`base`, `bash`, `bask`, `bass`, `bast`, `bath`, `bosh`, `bush`, `case`, `cash`, `cask`, `cast`, `dash`, `dish`, `gash`, `gasp`, `gosh`, `gush`, `hash`, `hasp`, `hath`, `hush`, `lash`, `lass`, `last`, `lath`, `lush`, `mash`, `mask`, `mass`, `mast`, `math`, `mesh`, `mush`, `push`, `ramp`, `rasp`, `ruse`, `rush`, `rusk`, `rust`, `sash`, `sass`, `tush`, `wash`, `wasp`, `wish`')

In [51]:
len(reachable_in('rash', 2)), ', '.join(sorted('<code>{}</code>'.format(r) for r in reachable_in('rash', 2)))

(47,
 '<code>base</code>, <code>bash</code>, <code>bask</code>, <code>bass</code>, <code>bast</code>, <code>bath</code>, <code>bosh</code>, <code>bush</code>, <code>case</code>, <code>cash</code>, <code>cask</code>, <code>cast</code>, <code>dash</code>, <code>dish</code>, <code>gash</code>, <code>gasp</code>, <code>gosh</code>, <code>gush</code>, <code>hash</code>, <code>hasp</code>, <code>hath</code>, <code>hush</code>, <code>lash</code>, <code>lass</code>, <code>last</code>, <code>lath</code>, <code>lush</code>, <code>mash</code>, <code>mask</code>, <code>mass</code>, <code>mast</code>, <code>math</code>, <code>mesh</code>, <code>mush</code>, <code>push</code>, <code>ramp</code>, <code>rasp</code>, <code>ruse</code>, <code>rush</code>, <code>rusk</code>, <code>rust</code>, <code>sash</code>, <code>sass</code>, <code>tush</code>, <code>wash</code>, <code>wasp</code>, <code>wish</code>')

In [31]:
len(reachable_in('rash', 3))

180

In [32]:
len(reachable_in('rash', 10))

2195

In [33]:
len(reachable_in('vice', 10))

2192

In [34]:
%%timeit
len(reachable_in('rash', 10))

100 loops, best of 3: 8.95 ms per loop


In [35]:
%%timeit
len(reachable_in('rash', 10, trim_extras=True))

100 loops, best of 3: 4.14 ms per loop


In [36]:
len(reachable_in('stay', 10))

2188

In [37]:
len(reachable_in('coax', 10))

2193

In [38]:
stay5 = reachable_in('stay', 4)
len(stay5)

280

In [39]:
extras = set()
for w in stay5:
    extras.update(neighbours[w])
extras.difference_update(stay5)
len(extras)

296

In [40]:
' '.join(extras)

'leak drab twin glib yell jets heed prep dram step grey keep sacs hock bloc bees hews bend chit sots rent knit brat bobs news gram boot baas grad beet door keen toot mewl plus dial bows jeep boom aloe fret need foul beep brew peek veep poop blue weep akin coop lead foot deed hoot teem quit chop teed bias moor sack atop went bier vets teen pout eery leaf drag belt crag peed czar gees veil whip bout jock wees twit beck boos tell vial pees brag tout sped weed geed load book crop text deem shes mead mock very sewn feet sows cell leis dell hemp week yens than mean send chin sill reek wiry knot reed crew peps pest stem melt room coup yews dock less suck boon clef emit bras loot boob pegs pews bran thin well geek boss term pled whir feed whom rock foam dent glue mews rood goat next sort tens goop stay soft perk troy held meet fees best lees reel gent nest brow spec fest coat leek sues pier moot knob test bogs boys said heft hell flee thaw clue prom self felt hair shoe airs cent whet pock gets

In [41]:
astar_search('coax', 'stay')

['coax', 'coat', 'boat', 'boar', 'soar', 'star', 'stay']

In [42]:
%time bfs_search_closed('coax', 'stay')

CPU times: user 2.35 s, sys: 0 ns, total: 2.35 s
Wall time: 2.35 s


['coax', 'coat', 'chat', 'shat', 'slat', 'slay', 'stay']

In [48]:
# %time bfs_search('coax', 'stay')

In [44]:
astar_search('czar', 'stay')

['czar', 'tzar', 'tear', 'sear', 'star', 'stay']

In [49]:
# %time bfs_search('czar', 'stay')

In [50]:
# %time bfs_search('coax', 'stay')